In [ ]:
import numpy as np
import h5py
import math
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import os, sys
import random
import scipy
from PIL import Image

In [ ]:
# set locations for working files
if len(sys.argv) != 2:
    print("Usage: python3 h5-to-img.py <automation_dir>")
    print('Assuming testing directories')
    automation_dir = '/mnt/analysis/e17023/Adam/GADGET2/'
else:
    # Automation directory
    automation_dir = '/mnt/analysis/e17023/Adam/GADGET2/' #sys.argv[1]

In [ ]:
class HiddenPrints:
	def __enter__(self):
		self._original_stdout = sys.stdout
		sys.stdout = open(os.devnull, 'w')

	def __exit__(self, exc_type, exc_val, exc_tb):
		sys.stdout.close()
		sys.stdout = self._original_stdout

def remove_outliers(xset, yset, zset, eset, threshold):
	"""
	Uses DBSCAN to find and remove outliers in 3D data
	"""
 
	data = np.array([xset.T, yset.T, zset.T]).T
 
	# STANDARD METHOD
	#DBSCAN_cluster = DBSCAN(eps=7, min_samples=10).fit(data)
	#out_of_cluster_index = np.where(DBSCAN_cluster.labels_==-1)

	# FISHTANK METHOD
	labels = dbscan(data, eps=7, min_samples=10)
	out_of_cluster_index = np.where(labels==-1)

	del data
	rev = out_of_cluster_index[0][::-1]
	#if len(out_of_cluster_index[0]) > 0:
	for i in rev:
		xset = np.delete(xset, i)
		yset = np.delete(yset, i)
		zset = np.delete(zset, i)
		eset = np.delete(eset, i)

	if len(xset) <= threshold:
		veto = True
	else:
		veto = False

	return xset, yset, zset, eset, veto

In [ ]:
def dbscan(X, eps, min_samples):
    # Initialize variables
    labels = np.zeros(X.shape[0])
    cluster = 0

    # Compute distances between points
    dists = np.sqrt(((X[:, np.newaxis] - X) ** 2).sum(axis=2))

    # Iterate over each point
    for i in range(X.shape[0]):
        # If point already visited, continue
        if labels[i] != 0:
            continue

        # Find neighboring points
        neighbors = np.where(dists[i] <= eps)[0]

        # If not enough neighboring points, label as noise
        if len(neighbors) < min_samples:
            labels[i] = -1
            continue

        # Expand cluster
        cluster += 1
        labels[i] = cluster

        while len(neighbors) > 0:
            j = neighbors[0]
            if labels[j] == -1:
                labels[j] = cluster
            elif labels[j] == 0:
                labels[j] = cluster
                new_neighbors = np.where(dists[j] <= eps)[0]
                if len(new_neighbors) >= min_samples:
                    neighbors = np.concatenate((neighbors, new_neighbors))
            neighbors = neighbors[1:]

    return labels

In [ ]:
def pca_func(data, n_components):
    
    # original method
    #pca = PCA(n_components=n_components)
    #pca.fit(data)
    #out = pca.transform(data)

    
    # fishtank-compatible method
    # Subtract the mean from the data to center it
    mean = np.mean(data, axis=0)
    data_centered = data - mean

    # Compute the covariance matrix
    cov = np.cov(data_centered, rowvar=False)

    # Compute the eigenvectors and eigenvalues of the covariance matrix
    eigvals, eigvecs = np.linalg.eig(cov)

    # Transform the data into the new coordinate system defined by the eigenvectors
    out = np.dot(data_centered, eigvecs)

    return out

In [ ]:
def track_len(xset, yset, zset):
    """
    Uses PCA to find the length of a track
    """
    veto_on_length = False
 
    # Form data matrix
    data = np.concatenate((xset[:, np.newaxis], 
                           yset[:, np.newaxis], 
                           zset[:, np.newaxis]), 
                           axis=1)

    # Use PCA to find track length
    principalComponents = pca_func(data, 3)
    
    principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2', 'principal component 3'])
    
    track_len = 2.35*principalDf.std()[0]
    if track_len > 70:
        veto_on_length = True
    
    # Force no veto for testing
    #veto_on_length = False
    
    angle = 0
    
    return track_len, veto_on_length, angle

In [ ]:
def main(h5file, pads, ic):
		"""
		This functions does the following: 
		- Converts h5 files into ndarrays. 
		- Removes outliers.
		- Calls PCA to return track length.
		- Sums mesh signal to return energy.
		"""
		# Converts h5 files into ndarrays, and output each event dataset as a separte list
		num_events = int(len(h5file.keys()))

		len_list = []
		good_events = []
		tot_energy = []
		trace_list = []
		xHit_list = []
		yHit_list = []
		zHit_list = []
		eHit_list = []
		angle_list = []

		cloud_missing = 0
		skipped_events = 0
		veto_events = 0

		#pbar = tqdm(total=num_events+1)
		for i in range(1, num_events+1):

			# Make copy of cloud datasets
			str_cloud = f"Event_[{i}]"
			try:
				cloud = np.array(h5file[str_cloud]['HitArray'])
			except:
				cloud_missing += 1
				#pbar.update(n=1)
				continue

			# Make copy of datasets
			cloud_x = cloud['x']
			cloud_y = cloud['y']
			cloud_z = cloud['z']
			#cloud_z = cloud[:,2] - np.min(cloud[:, 2])
			cloud_e = cloud['A']
			del cloud

			# Apply veto condition
			R = 36                           # Radius of the pad plane
			r = np.sqrt(cloud_x**2 + cloud_y**2)
			statements = np.greater(r, R)    # Check if any point lies outside of R

			if np.any(statements) == True:
				veto_events += 1
				#pbar.update(n=1)
				continue
			
			# Apply pad threshold
			x = (35 + cloud_x) * 2 + 42
			y = 145 - (35 + cloud_y) * 2
			xy_tuples = np.column_stack((x, y))
			unique_xy_tuples = set(map(tuple, xy_tuples))
			num_unique_tuples = len(unique_xy_tuples)

			if num_unique_tuples <= pads:
				skipped_events += 1
				#print(str_cloud, 'num_unique_tuples:', num_unique_tuples, '<= pads:', pads)
				#pbar.update(n=1)
				continue

			"""
			# Call remove_outliers to get dataset w/ outliers removed
			cloud_x, cloud_y, cloud_z, cloud_e, veto = remove_outliers(cloud_x, cloud_y, cloud_z, cloud_e, pads)
			if veto == True:
				skipped_events += 1
				pbar.update(n=1)
				continue
			"""
			# Move track next to pad plane for 3D view and scale by appropriate factor
			cloud_z = (cloud_z  - np.min(cloud_z ))*1.45 # Have also used 1.92
			#cloud_z = (cloud_z  - np.min(cloud_z ))

			# Call track_len() to create lists of all track lengths
			length, veto_on_length, angle = track_len(cloud_x, cloud_y, cloud_z)
			if veto_on_length == True:
				veto_events += 1
				#print(str_cloud, 'length:', length, '> 70')
				#pbar.update(n=1)
				continue 

			#str_trace = f"Trace_[{i}]"
			data_trace = np.array(h5file[str_cloud]['Trace'])
			# pad_nums = data_trace[:,4]			 

			trace = np.sum(data_trace[-512:], axis=0)
			del data_trace


			max_val = np.argmax(trace)
			low_bound = max_val - 75
			if low_bound < 0:
				low_bound = 5
			upper_bound = max_val + 75
			if upper_bound > 512:
				upper_bound = 506
			#trace = trace[low_bound:upper_bound]

			# Smooth trace
			#trace = smooth_trace(trace)

			# Subtract background and fit trace
			#polynomial_degree=poly 
			#baseObj=BaselineRemoval(trace)
			#trace=baseObj.IModPoly(polynomial_degree)

			# Remove noise, negative values, and zero consecutive bins
			#trace = remove_noise(trace, threshold_ratio=0.01)
	

			# Here you can apply the scale factor to the total energy
			scaled_energy = np.sum(trace)

			if scaled_energy > ic:
				veto_events += 1
				#print(str_cloud, 'scaled_energy:', scaled_energy, '> ic:', ic)
				#pbar.update(n=1)
				continue

			"""
			# Check to see if point is in "junk" region
			x1 = scaled_energy
			y1 = length
			y_line = slope * x1 + intercept
			if y1 > y_line and y1 > 20:
				veto_events += 1
				pbar.update(n=1)
				continue
			"""

			# Call track_angle to create list of all track angles
			angle_list.append(angle)

			# Append all lists
			len_list.append(length)
			tot_energy.append(scaled_energy)
			trace_list.append(trace)
			xHit_list.append(cloud_x)
			yHit_list.append(cloud_y)
			zHit_list.append(cloud_z)
			eHit_list.append(cloud_e)

			# Track original event number of good events
			good_events.append(i)
			#pbar.update(n=1)

		#print('Starting # of Events:', num_events)
		#print('Events Below Threshold:', skipped_events)
		#print('Vetoed Events:', veto_events)
		#print('Events Missing Cloud:', cloud_missing)
		#print('Final # of Good Events:', len(good_events))

		return (tot_energy, skipped_events, veto_events, good_events, len_list, trace_list, xHit_list, yHit_list, zHit_list, eHit_list, angle_list)

In [ ]:
def make_grid():
		"""
		"Create Training Data.ipynb"eate grid matrix of MM outline and energy bar, see spreadsheet below
		https://docs.google.com/spreadsheets/d/1_bbg6svfEph_g_Z002rmzTLu8yjQzuj_p50wqs7mMrI/edit?usp=sharing
		"""
		row = np.array([63, 47, 39, 31, 27, 23, 19, 15, 15, 11, 11, 7, 7, 7, 7, 3, 3, 3, 3, 3, 3, 3, 7, 7, 
			    7, 7, 11, 11, 15, 15, 19, 23, 27, 31, 39, 47, 63]) 

		to_row = np.array([87, 103, 111, 119, 123, 127, 131, 135, 135, 139, 139, 143, 143, 143, 143, 147, 
			       147, 147, 147, 147, 147, 148, 143, 143, 143, 144, 139, 140, 135, 136, 132, 128,
			       124, 120, 112, 104, 88]) 

		col = np.array([100, 84, 76, 68, 64, 60, 56, 52, 52, 48, 48, 44, 44, 44, 44, 40, 40, 40, 40, 40, 
			    40, 40, 44, 44, 44, 44, 48, 48, 52, 52, 56, 60, 64, 68, 76, 84, 100])

		to_col = np.array([124, 140, 148, 156, 160, 164, 168, 172, 172, 176, 176, 180, 180, 180, 180, 184, 
			       184, 184, 184, 184, 184, 184, 180, 180, 180, 180, 176, 176, 172, 172, 168, 164, 
			       160, 156, 148, 140, 124]) 

		all_row = np.array([i for i in range(3, 148, 4)])
		all_col = np.array([i for i in range(40, 185, 4)]) 

		full_image_size_width = 224
		full_image_size_length = 151
		mm_grid = np.zeros((full_image_size_length, full_image_size_width, 3))    
		mm_grid.fill(255)                                                     

		for i in range(len(row)):
			# draw grid columns, 0 = black
			mm_grid[row[i]:to_row[i], all_col[i], :] = 0
        
		for i in range(len(col)):
			# draw grid rows
			mm_grid[all_row[i], col[i]:to_col[i], :] = 0

		# Ensure that matrix is of integers
		mm_grid = mm_grid.astype(int) 

		# Draw energy bar box
		mm_grid = make_box(mm_grid)

		return mm_grid


def blue_range(pad_plane, rows):
	start_row = 140
	low_color = 0
	high_color = 35
	for i in range(rows):
		pad_plane[start_row:start_row+5, 8:17, 0] = low_color
		pad_plane[start_row:start_row+5, 8:17, 1] = high_color
		start_row = start_row - 5 
		low_color = low_color + 35
		high_color = high_color + 35
	return pad_plane
def yellow_range(pad_plane, rows):
	start_row = 105
	color = 220
	for i in range(rows):
		pad_plane[start_row:start_row+5, 8:17, 2] = color
		start_row = start_row - 5 
		color = color - 15
	return pad_plane
def orange_range(pad_plane, rows):
	start_row = 70
	color = 210
	for i in range(rows):
		pad_plane[start_row:start_row+5, 8:17, 1] = color - 15
		pad_plane[start_row:start_row+5, 8:17, 2] = color
		start_row = start_row - 5 
		color = color - 15
	return pad_plane
def red_range(pad_plane, rows):
	start_row = 35
	color = 250
	for i in range(rows):
		pad_plane[start_row:start_row+5, 8:17, 0] = color
		pad_plane[start_row:start_row+5, 8:17, 1] = 50
		pad_plane[start_row:start_row+5, 8:17, 2] = 50
		start_row = start_row - 5 
		color = color - 15
	return pad_plane
def tot_energy_to_mev(tot_energy):
	"""Convert tot_energy to energy in MeV using the line of best fit."""
	# Calculate the gradient and y-intercept for the line of best fit
	x1, y1 = 156745, 0.806 # Calibration values
	x2, y2 = 320842, 1.679
	gradient = (y2 - y1) / (x2 - x1)
	y_intercept = y1 - gradient * x1
	return gradient * tot_energy + y_intercept
def fill_energy_bar(pad_plane, tot_energy, event_energy):
	"""
	Fills the energy bar where the amount of pixels fired and the color corresponds to the energy of the track
	Max pixel_range should be 28 (7 rows for each color), so need to adjust accordingly.
	"""
	# Calculate the energy in MeV
	if event_energy != 0:
		energy_mev = event_energy/1000
	else:
		energy_mev = tot_energy_to_mev(tot_energy)
	# +/- 5% error in energy measurement
	energy_mev = energy_mev * random.uniform(0.95, 1.05)

	# Calculate the proportion of the energy bar that should be filled (scaled to 3 MeV max)
	proportion_filled = energy_mev / 3
	# Calculate how many rows should be filled
	total_rows = math.floor(proportion_filled * 28)
	# Fill the energy bar one row at a time
	if total_rows > 0:
		pad_plane = blue_range(pad_plane, rows=min(total_rows, 7))
	if total_rows > 7:
		pad_plane = yellow_range(pad_plane, rows=min(total_rows-7, 7))
	if total_rows > 14:
		pad_plane = orange_range(pad_plane, rows=min(total_rows-14, 7))
	if total_rows > 21:
		pad_plane = red_range(pad_plane, rows=min(total_rows-21, 7))
	return pad_plane
def pos_odd_even(event_value):
	"""
	Makes correction to positive points if they are odd or even
	"""
	if event_value % 2 == 0:
		event_value = event_value + 1
		return event_value
	else:
		return event_value
def neg_odd_even(event_value):
	"""
	Makes correction to negative points if they are odd or even
	"""
	if event_value % 2 == 0:
		event_value = event_value - 1
		return event_value
	else:
		return event_value
def pt_shift(xset, yset):
	"""
	Shifts all points to the center of nearest pad for pad mapping
	"""
	for j in range(len(xset)):
		if xset[j] > 0:
			xset[j] = math.floor(xset[j])
			pos_adj_valx = pos_odd_even(xset[j])
			xset[j] = pos_adj_valx
		elif xset[j] < 0:
			xset[j] = math.ceil(xset[j])
			neg_adj_valx = neg_odd_even(xset[j])
			xset[j] = neg_adj_valx
		if yset[j] > 0:
			yset[j] = math.floor(yset[j])
			pos_adj_valy = pos_odd_even(yset[j])
			yset[j] = pos_adj_valy
		elif yset[j] < 0:
			yset[j] = math.ceil(yset[j])
			neg_adj_valy = neg_odd_even(yset[j])
			yset[j] = neg_adj_valy
	return xset, yset
def make_box(mm_grid):
	"""
	Draws the box for the energy bar
	"""
	box_row = np.array([4, 4])
	to_box_row = np.array([145, 146])
	for_box_col = np.array([7, 17])
	box_col = np.array([7, 7])
	to_box_col = np.array([17, 17])
	for_box_row = np.array([4, 145])
	# Draw vertical lines of energy bar box
	for i in range(len(box_row)):
		mm_grid[box_row[i]:to_box_row[i], for_box_col[i], :] = 0
		mm_grid[for_box_row[i], box_col[i]:to_box_col[i], :] = 0
	return mm_grid
def fill_padplane(xset, yset, eset, trace, event_energy):
	"""
	Fills the 2D pad plane grid for image creation
	"""
	pad_plane = make_grid()
	xset = np.array(xset)
	yset = np.array(yset)
	eset = np.array(eset)
	# pad plane mapping
	x = (35 + xset) * 2 + 42    # col value
	y = 145 - (35 + yset) * 2   # row value
	# create a dictionary to store (x,y) as keys and e as values
	d = {}
	for i in range(len(x)):
		key = (x[i], y[i])
		if key in d:
			d[key] += eset[i]
		else:
			d[key] = eset[i]
	# convert the dictionary back to arrays
	x = np.zeros(len(d))
	y = np.zeros(len(d))
	eset = np.zeros(len(d))
	for i, key in enumerate(d):
		x[i] = key[0]
		y[i] = key[1]
		eset[i] = d[key]
	    
	# Find max E value and normalize
	energy = eset
	max_energy = np.max(energy)
	norm_energy = energy / max_energy
	# Fill in pad plane   
	for k in range(len(x)):
	
		if y[k] < 9:
			y[k] = y[k] + 4
		if x[k] < 50:
			x[k] = x[k] + 4
		if x[k] > 174:
			x[k] = x[k] - 4
		if y[k] > 53:
			y[k] = y[k] - 4
		if x[k] > 134:
			x[k] = x[k] - 4
		if y[k] > 93:
			y[k] = y[k] - 4
		if y[k] > 133:
			y[k] = y[k] - 4	
		if x[k] < 90:
			x[k] = x[k] + 4
		pad_plane[int(y[k])-1:int(y[k])+2, int(x[k])-1:int(x[k])+2, 0] = norm_energy[k] * 205
		pad_plane[int(y[k])-1:int(y[k])+2, int(x[k])-1:int(x[k])+2, 1] = norm_energy[k] * 240
	pad_plane = fill_energy_bar(pad_plane, trace.sum(), event_energy)
	return pad_plane
def trace_image(padplane_image, trace):
	#Creates a 2D image from trace data
	# Save plot as jpeg (only want RGB channels, not an alpha channel)
	# Need to take monitor dpi into account to get correct pixel size
	# Plot should have a pixel size of 73x224
	my_dpi = 96
	fig, ax = plt.subplots(figsize=(224/my_dpi, 73/my_dpi))
	max_val = np.argmax(trace)
	low_bound = max_val - 75
	if low_bound < 0:
		low_bound = 5
	upper_bound = max_val + 75
	if upper_bound > 511:
		upper_bound = 506
	trace = trace[low_bound:upper_bound]
	x = np.linspace(0, len(trace)-1, len(trace))
	
	ax.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)
	ax.spines['top'].set_visible(False)
	ax.spines['right'].set_visible(False)
	ax.spines['bottom'].set_visible(False)
	ax.spines['left'].set_visible(False)
	ax.fill_between(x, trace, color='b', alpha=1)
	rand_num = random.randrange(0,1000000,1)
	temp_strg = f'/mnt/analysis/e17023/Adam/GADGET2/simOutput/energy_depo_{rand_num}.jpg'
	plt.savefig(temp_strg, dpi=my_dpi)
	plt.close()
	# Load png plot as a matrix so that it can be appended to pad plane plot
	img = plt.imread(temp_strg)
	os.remove(temp_strg)
	rows,cols,colors = img.shape # gives dimensions for RGB array
	img_size = rows*cols*colors
	img_1D_vector = img.reshape(img_size)
	# you can recover the orginal image with:
	trace_image = img_1D_vector.reshape(rows,cols,colors)
	# append pad plane image with trace image
	complete_image = np.append(padplane_image, trace_image, axis=0)
	return complete_image

In [ ]:
def track_main(h5file, threshold, event_energy):
    """
    This functions does the following: 
    - Converts h5 files into ndarrays. 
    - Maps the column values (x) and row values (y) from the h5 files into a 3 x 224 x 224 matrix. 
    - Uses charge per pad to encode RGB colors. 
    - Creates an image of 2D pad plane for each event. 
    """
    # Converts h5 files into ndarrays, and output each event dataset as a separte list
    num_events = int(len(h5file))
    mm_grid = make_grid()
    pad_plane = np.repeat(mm_grid[np.newaxis, :, :], 1, axis=0)
    new_pad_plane = np.repeat(mm_grid[np.newaxis, :, :], 1, axis=0)
    complete_image = []

    new_good_events = []
    skipped_events = 0
    veto_events = 0
    cut_events = 0
    
    # pbar = tqdm(total=num_events)
    for i in range(0, num_events):
            
        str_event = f"Event_[{i}]"
        str_trace = f"Trace_[{i}]"
        
        # Apply pad threshold
        event = np.array(h5file[str_event]['HitArray'][:])
        trace = np.array(h5file[str_event]['Trace'][:])

        # skip event if length is 0
        if len(h5file[str_event]) == 0:
            #print('skipped event', i, 'because length is 0')
            skipped_events += 1
            continue
        if len(event) < threshold:
            #print('skipped event', i, 'because pads hit is < threshold:', threshold)
            skipped_events += 1
            continue

        # Make copy of datasets
        dset_0_copyx = event['x']
        dset_0_copyy = event['y']
        dset_0_copyz = event['z'] - min(event['z'])
        dset_0_copye = event['A']
            
        # Track event number of good events
        new_good_events.append(i)
            
        # Call pt_shift function to move all 2D pts to pad centers
        dset_0_copyx, dset_0_copyy = pt_shift(dset_0_copyx, dset_0_copyy)
                
        # Call fill_padplane to create 2D pad plane image
        pad_plane = np.append(pad_plane, new_pad_plane, axis=0)
        pad_plane[i-skipped_events-veto_events-cut_events] = fill_padplane(dset_0_copyx, dset_0_copyy, dset_0_copye, trace, event_energy)
        
        # Call trace_image() to append trace to pad plane image
        complete_image.append(trace_image(pad_plane[i-skipped_events-veto_events-cut_events], trace))
        
        # pbar.update(n=1)
        
    return (complete_image, skipped_events, veto_events, new_good_events, cut_events)

In [ ]:
# H5 file to read
h5_path = automation_dir + "simOutput/hdf5/"
image_path = automation_dir + "simOutput/images/"
param_file = automation_dir + "simOutput/parameters.csv"


h5_list = os.listdir(h5_path)
param_df = pd.read_csv(param_file)

for file in h5_list:
    if file.endswith(".h5"):
        hdf5_name = file[:-3]
        #print(hdf5_name)
    else:
        continue
    f = h5py.File(h5_path + hdf5_name + '.h5', 'r')
    h5file = f
    
    # Get energy from parameters file (if it exists)
    try:
        event_energy = param_df.loc[param_df['Sim'] == hdf5_name, 'E0'].iloc[0] + param_df.loc[param_df['Sim'] == hdf5_name, 'E1'].iloc[0]
    except:
        event_energy = 0
    
    # Call function to create lists
    (tot_energy, skipped_events, veto_events, good_events, len_list, trace_list, xHit_list, yHit_list, zHit_list, eHit_list, angle_list) = main(h5file=f, pads=2, ic=99999999)
    
    # Call function to create images
    (image, skipped_events, veto_events, new_good_events, cut_events) = track_main(h5file=h5file, threshold=2, event_energy=event_energy)
    
    i = 0
    for img_num in range(len(new_good_events)):
        if new_good_events[img_num] in good_events:
            i += 1
            plt.imsave(image_path+hdf5_name+"_"+str(new_good_events[img_num])+".png", image[img_num])
    print(hdf5_name, ':', i, "good images from", len(h5file), "events")
    h5file.close()

In [ ]:
# CREATE GIFS FROM IMAGES
h5_dir = automation_dir+"simOutput/hdf5/"
img_dir = automation_dir+"simOutput/images/"
gif_dir = automation_dir+"simOutput/gifs/"

imagelist = [ f for f in os.listdir(img_dir) if f.endswith(".png")]
h5_list = [f.split(".h")[0] for f in os.listdir(h5_dir)]

for h5 in h5_list:
    filt_imglist = [f for f in imagelist if h5 in f]
    # create gif from images in filelist
    frames = []
    for i in range(len(filt_imglist)):
        new_frame = Image.open(img_dir+filt_imglist[i])
        frames.append(new_frame)
    
    # Save into a GIF file
    try:
        frames[0].save(f'{gif_dir}{h5}.gif', format='GIF', append_images=frames[1:], save_all=True, duration=250, loop=0)
    except:
        print(f"Error converting {h5} images to gif")
        continue